In [2]:
import pandas as pd
import numpy as np
import os
import glob
from reference.atminter.lib.modules.paperparse import spFile

# Align and check dataset from @MInter
## Info reported in supplementary files

In [124]:
# Core dataset
core_dataset = {
    'core_pair': ('lactobacillus acidophilus', 'escherichia coli')
}
xl = pd.ExcelFile('/home/yuning/Work/cs_273/reference/atminter/supplement_info/Supplementary_file_1_Core_Dataset_PMIDs.xlsx')
core_dataset['abstract_labels'] = xl.parse(xl.sheet_names[0])

print("Core dataset:\tCore pair:{}\n\t\tAbstract number:{}\n\t\tw/ interaction:{}".format(
    core_dataset['core_pair'],
    len(core_dataset['abstract_labels']),
    np.sum(core_dataset['abstract_labels']['Interaction'])
))

# Extended dataset
ext_dataset = {}
xl = pd.ExcelFile('/home/yuning/Work/cs_273/reference/atminter/supplement_info/Supplementary_file_2_Extended_Dataset_Interactions.xlsx')
ext_dataset['interaction_labels'] = xl.parse(xl.sheet_names[0])

xl = pd.ExcelFile('/home/yuning/Work/cs_273/reference/atminter/supplement_info/Supplementary_file_3_Extended_Dataset_PMIDs.xlsx')
ext_dataset['abstract_labels'] = xl.parse(xl.sheet_names[0])

print('Ext. dataset:\tInteraction pairs(may contain duplicates):{}\n\t\tLabeled abstracts:{}'.format(
    len(ext_dataset['interaction_labels']),
    len(ext_dataset['abstract_labels'])
))

# pattern scan
xl = pd.ExcelFile('/home/yuning/Work/cs_273/reference/atminter/supplement_info/Supplementary_file_4_Patterns_and_Terms.xlsx')
scan_patterns = xl.parse(xl.sheet_names[0])
scan_patterns

# skin data set
skin_dataset = {}
xl = pd.ExcelFile('/home/yuning/Work/cs_273/reference/atminter/supplement_info/Supplementary_file_5_List_of_Skin_Bacteria.xlsx')
skin_dataset['species'] = xl.parse(xl.sheet_names[0])
xl = pd.ExcelFile('/home/yuning/Work/cs_273/reference/atminter/supplement_info/Supplementary_file_6_Curated_Interactions_between_Skin_Bacteria.xlsx')
skin_dataset['curated_interaction'] = xl.parse(xl.sheet_names[0])

Core dataset:	Core pair:('lactobacillus acidophilus', 'escherichia coli')
		Abstract number:350
		Interaction:79
Ext. dataset:	Interaction pairs(may contain duplicates):62
		Labeled abstracts:3567


In [130]:
np.sum(ext_dataset['abstract_labels']['Interaction'] == 0)

3405

79

## Info from Extended dataset ```/data/train_test_data/collated_annotations```

In [85]:
import glob

collated_data_root = '/home/yuning/Work/cs_273/reference/atminter/data/train_test_data/collated_annotations/'
sample_set = []
pair_abstract_set = {}
for set_num in [1, 2, 3, 4, 5]:
    files = glob.glob(collated_data_root + 'set_{}/*.sp'.format(set_num))
    print("set_{}: number of .sp files: {}".format(set_num, len(files)))
    for file in files:
        sp_file = spFile(spFilePath=file)
        pair_abstract_set[tuple(sp_file.species_names)] = sp_file

set_1: number of .sp files: 155
set_2: number of .sp files: 150
set_3: number of .sp files: 153
set_4: number of .sp files: 135
set_5: number of .sp files: 153


In [90]:
print('Number of pair files: {}'.format(len(pair_abstract_set)))

Number of pair files: 735


In [101]:
papers_PMID_set = set()
for sp_file in pair_abstract_set.values():
    papers_PMID_set.update([paper['PMID'] for paper in sp_file.papers])
print('Number of abstracts with differetn PMID: {}'.format(len(papers_PMID_set)))

Number of abstracts with differetn PMID: 29948


In [131]:
sp_file.papers

[{'AB  ': 'the objective of this study was to characterize the subgingival microbiota of african-american children with localized aggressive periodontitis (lap). fifty-one children were included. subgingival plaque samples were taken from diseased (dd) and healthy sites (dh) in lap and from healthy sites in hs and hc and analyzed by 16s rrna-based microarrays. aggregatibacter actinomycetemcomitans (aa) was the only species found to be both more prevalent (or = 8.3, p = 0.0025)  and abundant (p < 0.01) in dd. filifactor alocis (fa) was also found to be more prevalent in dd (or 2.31, ci 1.06-5.01, p = 0.03). most prevalent species in healthy sites were selenomonas spp, veillonella spp, streptococcus spp, bergeyella sp, and kingella oralis. overall, streptococcus spp, campylobacter gracilis, capnocytophaga granulosa, haemophilus parainfluenzae, and lautropia mirabilis were most abundant in healthy children, while aa, fa, tannerella sp, solobacterium moorei, parvimonas micra, and capnocyto

In [32]:
sp_file.species_names

['Lactobacillus crispatus', 'Gardnerella vaginalis']

set_1: number of .sp files: 155
set_2: number of .sp files: 150
set_3: number of .sp files: 153
set_4: number of .sp files: 135
set_5: number of .sp files: 153


In [64]:
all_pairs = []
for sample in sample_set:
    sample['pairs'] = [tuple(pair) for pair in sample['pairs']]
    all_pairs += sample['pairs']

In [73]:
import numpy as np
import pprint

repeat_pairs = [(pair,[pair in sample['pairs'] for sample in sample_set]) for pair in set(all_pairs)
                if np.sum([pair in sample['pairs'] for sample in sample_set]) > 1]
pprint.pprint(repeat_pairs)

[(('Streptococcus cristatus', 'Capnocytophaga gingivalis'),
  [True, True, False, False, False]),
 (('Staphylococcus aureus', 'Desulfovibrio desulfuricans'),
  [True, False, True, False, False]),
 (('Bacteroides caccae', 'Faecalibacterium prausnitzii'),
  [False, True, False, True, False]),
 (('Bacteroides uniformis', 'Bacteroides coprocola'),
  [False, False, True, True, False]),
 (('Streptococcus oralis', 'Haemophilus influenzae'),
  [False, False, True, True, False]),
 (('Ruminococcus flavefaciens', 'Fusobacterium gonidiaformans'),
  [True, True, False, False, False]),
 (('Zymomonas mobilis', 'Propionibacterium freudenreichii'),
  [True, False, True, False, False]),
 (('Lactobacillus amylovorus', 'Lactobacillus casei'),
  [False, True, True, False, False]),
 (('Yersinia enterocolitica', 'Fusobacterium varium'),
  [False, False, True, True, False]),
 (('Atopobium vaginae', 'Bacteroides fragilis'),
  [True, False, True, False, False]),
 (('Paracoccus denitrificans', 'Salmonella enteri

In [82]:
file_1 = collated_data_root + 'set_3/Yersinia_enterocolitica#Fusobacterium_varium.sp'
file_2 = collated_data_root + 'set_4/Yersinia_enterocolitica#Fusobacterium_varium.sp'

spFile_1 = spFile(spFilePath=file_1)
spFile_2 = spFile(spFilePath=file_2)

In [83]:
pprint.pprint([paper['PMID'] for paper in spFile_1.papers])
pprint.pprint([paper['PMID'] for paper in spFile_2.papers])

['2875188']
['2875188']
